In [1]:
import numpy as np
import pandas as pd
import pickle
import os

In [2]:
# Add the python path to the folder containing some useful custom packages.
import sys
sys.path.insert(0, "../packages/")
from LagsCreator.LagsCreator import LagsCreator
from NestedCV.NestedCV import NestedCV

In [3]:
# Create the workspace folder for storing training and test points.
dir_data = "./data_xgboost"
os.makedirs(dir_data)

In [4]:
# Select the countries to consider for the creation of training and test points.
COUNTRIES = ["Yemen", "Syria"]
# Define the name of indicator we want to predict.
TARGET = "FCS"
# Define the number of days we want to learn to predict for the target variable.
TEST_SIZE = 30
# Define the number of total split we want to evaluate using our nested cross validation method.
NUMBER_OF_SPLITS = 7
# Define the time features we want to create for the input samples.
FEATURE_TIMES = ["Day", "Month", "Dayofweek", "Year"]
# Define the step between points during the creation of samples for training and test.
STEP_BETWEEN_POINTS = 1

In [5]:
# Save the parameters.
with open(dir_data + "/global_variables", "wb") as f:
    pickle.dump([TARGET, TEST_SIZE, FEATURE_TIMES, COUNTRIES, NUMBER_OF_SPLITS, STEP_BETWEEN_POINTS], f)

## Time-series dataset

In [6]:
# Load the time-series data of the Yemen country.
df_yemen = pd.read_csv("../Dataset time-series/output_data/Yemen/Yemen.csv", header = [0, 1], index_col = 0)
df_yemen.index.name = "Datetime"
df_yemen.index = pd.to_datetime(df_yemen.index)
freq = "D"
df_yemen.index.freq = freq
df_yemen.columns = pd.MultiIndex.from_tuples(map(lambda x: ("Yemen", x[0], x[1]), df_yemen.columns), names = ["Country", "AdminStrata", "Indicator"])

In [7]:
# Load the time-series data of the Syria country.
df_syria = pd.read_csv("../Dataset time-series/output_data/Syria/Syria.csv", header = [0, 1], index_col = 0)
df_syria.index.name = "Datetime"
df_syria.index = pd.to_datetime(df_syria.index)
freq = "D"
df_syria.index.freq = freq
df_syria.columns = pd.MultiIndex.from_tuples(map(lambda x: ("Syria", x[0], x[1]), df_syria.columns), names = ["Country", "AdminStrata", "Indicator"])

In [8]:
# Concatenate data.
df = pd.concat([df_yemen, df_syria], axis = 1)
# Delete the feature 'Milk and dairy' that it isn't presents for the Yemen country.
df = df.drop("Milk and dairy", axis = 1, level = 2)
# Consider the following dates.
df = df.loc["2018-01-01":"2020-08-31"]
# Select countries.
df = df[COUNTRIES]
df.head()

Country                            Yemen                                \
AdminStrata                        Abyan                                 
Indicator   1 Month Anomaly (%) Rainfall 3 Months Anomaly (%) Rainfall   
Datetime                                                                 
2018-01-01                           NaN                           NaN   
2018-01-02                           NaN                           NaN   
2018-01-03                           NaN                           NaN   
2018-01-04                           NaN                           NaN   
2018-01-05                           NaN                           NaN   

Country                                                              \
AdminStrata                                                           
Indicator   Cereals and tubers Code Exchange rate FCS Fatality Fuel   
Datetime                                                              
2018-01-01                 NaN   65           NaN NaN      NaN  NaN   
2018-01-02                 NaN   65           NaN NaN      NaN  NaN   
2018-01-03                 NaN   65           NaN NaN      NaN  NaN   
2018-01-04                 NaN   65           NaN NaN      NaN  NaN   
2018-01-05                 NaN   65           NaN NaN      NaN  NaN   

Country                            ...   Syria                            \
AdminStrata                        ... Tartous                             
Indicator          Lat        Lon  ...    NDVI NDVI Anomaly Oil and fats   
Datetime                           ...                                     
2018-01-01   13.704878  46.158142  ...     NaN          NaN          NaN   
2018-01-02   13.704878  46.158142  ...     NaN          NaN          NaN   
2018-01-03   13.704878  46.158142  ...     NaN          NaN          NaN   
2018-01-04   13.704878  46.158142  ...     NaN          NaN          NaN   
2018-01-05   13.704878  46.158142  ...     NaN          NaN          NaN   

Country                                                       \
AdminStrata                                                    
Indicator   Population Pulses and nuts Rainfall (mm) Ramadan   
Datetime                                                       
2018-01-01      831296             NaN           NaN     NaN   
2018-01-02      831296             NaN           NaN     NaN   
2018-01-03      831296             NaN           NaN     NaN   
2018-01-04      831296             NaN           NaN     NaN   
2018-01-05      831296             NaN           NaN     NaN   

Country                                      
AdminStrata                                  
Indicator   Vegetables and fruits Wage rCSI  
Datetime                                     
2018-01-01                    NaN  NaN  NaN  
2018-01-02                    NaN  NaN  NaN  
2018-01-03                    NaN  NaN  NaN  
2018-01-04                    NaN  NaN  NaN  
2018-01-05                    NaN  NaN  NaN  

[5 rows x 682 columns]

## Creation training and test sets

In [9]:
# Define lags dictionary for each indicator.
lags_dict = dict()
# Define lags for each indicator.
lags_dict["3 Months Anomaly (%) Rainfall"] = None
lags_dict["1 Month Anomaly (%) Rainfall"] = np.array([1,2,3]) 
lags_dict["Rainfall (mm)"] = np.array([1,2,3,4,5,6]) 
lags_dict["Exchange rate"] = np.array([1,2,3]) 
lags_dict["Cereals and tubers"] = np.array([1,2,3]) 
lags_dict["Fuel"] = None
lags_dict["Meat, fish and eggs"] = None
lags_dict["Miscellaneous food"] = None
lags_dict["Oil and fats"] = None
lags_dict["Pulses and nuts"] = None
lags_dict["Vegetables and fruits"] = None
lags_dict["Wage"] = None
lags_dict["Fatality"] = np.arange(1, 61)
lags_dict["NDVI Anomaly"] = np.array([1,2,3]) 
lags_dict["NDVI"] = None
lags_dict["FCS"] = np.arange(1, 61)
lags_dict["rCSI"] = np.arange(1, 61)
lags_dict["Lat"] = np.array([1])
lags_dict["Lon"] = np.array([1])
lags_dict["Population"] = np.array([1])
lags_dict["Code"] = np.array([1])
lags_dict["Ramadan"] = np.array([1])

In [10]:
# Save the lags dictionary.
with open(dir_data + "/lags_dict", "wb") as fp:
    pickle.dump(lags_dict, fp)

In [15]:
# Create folder for containing training data.
os.makedirs(dir_data + "/train")
# Create folder for containing test data.
os.makedirs(dir_data + "/test")
for country in COUNTRIES:
    provinces = df[country].columns.get_level_values(0).unique()
    for province in provinces:
        os.makedirs(dir_data + "/train/%s/%s" % (country, province)) 
        os.makedirs(dir_data + "/test/%s/%s" % (country, province)) 

In [11]:
# Create the nested cross validation.
cv = NestedCV(NUMBER_OF_SPLITS, TEST_SIZE)
# Total nested cross validation.
SPLITS = cv.get_splits(df)
for split_number, (train, test) in SPLITS.items():
    print("Split %d: range of days to predict between %s - %s" % (split_number, str(test.index[0].date()), str(test.index[-1].date())))

Split 1: range of days to predict between 2020-02-01 - 2020-03-01
Split 2: range of days to predict between 2020-03-01 - 2020-03-30
Split 3: range of days to predict between 2020-04-01 - 2020-04-30
Split 4: range of days to predict between 2020-05-01 - 2020-05-30
Split 5: range of days to predict between 2020-06-01 - 2020-06-30
Split 6: range of days to predict between 2020-07-01 - 2020-07-30
Split 7: range of days to predict between 2020-08-01 - 2020-08-30


In [16]:
for split_number, (train, test) in SPLITS.items():
    print("Split %d. Please wait." % split_number)
    # Define the first multi-sites (countries).
    countries = train.columns.get_level_values(0).unique()
    for country in countries:
        train_country = train[country]
        # Define the second multi-sites (provinces).
        provinces = train_country.columns.get_level_values(0).unique()
        for province in provinces:
            creator = LagsCreator(train_country[province], lags_dictionary = lags_dict, target = TARGET, delay = True)
            for h in range(TEST_SIZE):
                # Training samples.
                X_train, y_train, X_test, features = creator.to_supervised(h = h+1, step = STEP_BETWEEN_POINTS, single_step = True, 
                                                                           return_dataframe = True, feature_time = FEATURE_TIMES, 
                                                                           dtype = float)

                # Train input and output.
                X_train.to_csv(dir_data + "/train/%s/%s/X_train_split%d_h%d.csv" % (country, province, split_number, h+1), index_label = False) 
                y_train.to_csv(dir_data + "/train/%s/%s/y_train_split%d_h%d.csv" % (country, province, split_number, h+1), index_label = False) 
                # Test input.
                X_test.to_csv(dir_data + "/test/%s/%s/X_test_split%d_h%d.csv" % (country, province, split_number, h+1), index_label = False) 

print("Complete!")

Split 1. Please wait.
Split 2. Please wait.
Split 3. Please wait.
Split 4. Please wait.
Split 5. Please wait.
Split 6. Please wait.
Split 7. Please wait.
Complete!
